# Proyecto DL

### Librerias

In [1]:
from transformers import MarianMTModel, MarianTokenizer

In [8]:
import pandas as pd

In [10]:
!pip install rouge

In [11]:
from rouge import Rouge

In [25]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.4 MB/s eta 0:00:00


In [26]:
import sacrebleu

### Modelo y función de translate

https://huggingface.co/Helsinki-NLP/opus-mt-es-en

In [2]:
model_name = 'Helsinki-NLP/opus-mt-es-en'

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [37]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [3]:
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    translated = model.generate(**inputs)

    translation = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translation

In [7]:
input = "He aprendido mucho sobre modelos de lenguaje en este proyecto"
r = "I have learned a lot about language models in this project"
c = translate(input)

print("Traducción al inglés:", c)

Traducción al inglés: I've learned a lot about language models in this project.


### Metrica Rouge

https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499

https://github.com/pltrdy/rouge

R: reference

C: candidate


In [12]:
rouge = Rouge()

In [13]:
scores = rouge.get_scores(c, r)

In [14]:
scores

[{'rouge-1': {'r': 0.8181818181818182, 'p': 0.9, 'f': 0.8571428521541952},
  'rouge-2': {'r': 0.8, 'p': 0.8888888888888888, 'f': 0.8421052581717452},
  'rouge-l': {'r': 0.8181818181818182, 'p': 0.9, 'f': 0.8571428521541952}}]

### Metrica bleu

https://aclanthology.org/P02-1040.pdf

In [34]:
bleu_score = sacrebleu.corpus_bleu([c], [[r]])

In [35]:
bleu_score.score

78.60753021519781

In [ ]:
# import sacrebleu

# # Función para calcular BLEU Score
# def calculate_bleu(reference_texts, candidate_texts):
#     bleu_score = sacrebleu.corpus_bleu(candidate_texts, [reference_texts])
#     return bleu_score.score

# # Extracción de las columnas necesarias
# reference_texts = data['reference'].tolist()
# candidate_texts = data['candidate'].tolist()

# # Cálculo del BLEU Score
# bleu_score = calculate_bleu(reference_texts, candidate_texts)
# print(f"BLEU Score: {bleu_score}")

### Dataset

https://www.dropbox.com/scl/fi/xtvls58le65vxjfkmuv9x/Sentence-pairs-in-Spanish-English-2024-07-03.tsv?rlkey=3la4vmua8d5f5ltdknfhmcmyt&st=6cdj4k63&dl=0

Nose que significa ese number, en vola id de la traduccion tb?

In [18]:
column_names = ['id_reference', 'reference', 'number', 'candidate']

data = pd.read_csv('/content/Sentence pairs in Spanish-English - 2024-07-03.tsv', sep='\t', names=column_names, header=None)

In [36]:
data.head()

,id_reference,reference,number,candidate
0,2481,¡Intentemos algo!,1276.0,Let's try something.
1,2482,Tengo que irme a dormir.,1277.0,I have to go to sleep.
2,2483,¿Qué estás haciendo?,16492.0,What are you doing?
3,2483,¿Qué estás haciendo?,241947.0,What are you doing now?
4,2483,¿Qué estás haciendo?,516639.0,What are you up to?


In [24]:
data.describe()

,id_reference,number
count,2.652430e+05,2.652420e+05
mean,3.848433e+06,3.124278e+06
std,3.394753e+06,3.462222e+06
min,2.481000e+03,1.276000e+03
25%,1.278956e+06,3.277772e+05
50%,2.202354e+06,1.812746e+06
75%,5.814020e+06,4.597432e+06
max,1.254635e+07,1.254635e+07


### NER(Named-entity recognition)

https://github.com/marcelomendoza/IIC3670/blob/main/codes/8%20-%20SEQUENCE%20LABELING%2C%20POS%20TAGGING%20Y%20NER.ipynb